In [1]:
from brian2 import *
import numpy as np
import warnings

#warnings.filterwarnings("ignore", category=UserWarning, module='brian2.codegen.generators.base')

start_scope()

defaultclock.dt = 0.0001*ms  

# Custom timing function
@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, result=1, sum=1, spikes_received=1)
def spike_timing(w, global_clock, layer, sum, spikes_received): 
    x = global_clock % 1
    if w >= 0:
        return (x ** (1 - w)) 
    else:
        return (1 - (1 - x) ** (1 + w)) 
    
@implementation('numpy', discard_units=True)
@check_units(layer=1, result=1, sum=1, spikes_received=1)
def math1(layer, sum, spikes_received): 
    return (sum/spikes_received )+ layer


In [ ]:
# Improved gradient descent for SNN: ensure gradients are calculated and weights updated correctly

def improved_update_last_layer_weights(spike_times, desired_outputs, w_3, learning_rate=0.1):
    """
    Update output weights using softmax + cross-entropy.
    w_3: shape 30 (flattened 10 hidden * 3 outputs)
    """
    hidden_activations = np.array(spike_times[4:14])   # 10 hidden neurons
    # Build weight matrix (3 classes x 10 hidden units)
    weights = np.zeros((3, 10))
    for out_idx in range(3):
        weights[out_idx] = w_3[out_idx::3]

    # Forward Pass: logits and softmax
    logits = weights @ hidden_activations
    probs = np.exp(logits - np.max(logits))
    probs /= np.sum(probs)

    # Loss (for monitoring)
    loss = -np.sum(np.array(desired_outputs) * np.log(probs + 1e-12))

    # Gradient: dL/dz = softmax_output - target
    grad_logits = probs - np.array(desired_outputs)

    # Gradient: dL/dW = outer(grad_logits, hidden_activations)
    grad_weights = np.outer(grad_logits, hidden_activations)

    # Weight Update
    weights -= learning_rate * grad_weights

    # Clip (optional)
    weights = np.clip(weights, -5.0, 5.0)

    # Flatten back into w_3 list (stride write-back)
    for out_idx in range(3):
        for i in range(10):
            w_3[out_idx + i * 3] = weights[out_idx, i]

    # Diagnostics
    print("Updated weights (first 6):", w_3[:6])
    print("Gradient signs:", np.sign(grad_weights))
    print("Softmax probabilities:", probs)
    print("Cross-entropy loss:", loss)
    return w_3

# Example usage:
# w_3 = improved_update_last_layer_weights(var, wanted_output, w_3)

In [2]:
# # Urd, Verdande, Skuld 

# did not apply the specficyed weifhts to the first layer yet will maybe do after
def run_Urd(inputs, weights_1, weights_2, weights_3):
    '''4-10-3 SNN'''
    # will add check of weights # so it all works
    n_input = 4 
    n_hidden = 10
    n_output = 3
    n_total = n_input + n_hidden + n_output

    neurons = NeuronGroup(n_total, '''
        v : 1
        sum : 1
        spikes_received : 1
        scheduled_time : second
        global_clock : 1
    ''', threshold='v > 1', reset='v = 0', method='exact')
    neurons.v = 0
    neurons.scheduled_time = 1e9 * second
    neurons.global_clock = 0.0
    neurons.sum = 0.0
    neurons.spikes_received = 0.0


    indicess = [i for i in range(n_input)]
    stim = SpikeGeneratorGroup(n_input, indices=indicess, times=(inputs*ms))

    syn_input = Synapses(stim, neurons[0:n_input], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_input.connect(j='i')
    syn_input.w = weights_1
    syn_input.layer = 0

    syn_hidden = Synapses(neurons[0:n_input], neurons[n_input:n_input+n_hidden], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    for inp in range(n_input):
        for hid in range(n_hidden):
            syn_hidden.connect(i=inp, j=hid)

    syn_hidden.w = weights_2
    syn_hidden.layer = 1


    syn_output = Synapses(
        neurons[n_input:n_input+n_hidden], 
        neurons[n_input+n_hidden:n_total], 
        '''
        w : 1
        layer : 1
        ''',
        on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
        '''
    )

    for hid in range(n_hidden):
        for out in range(n_output):
            syn_output.connect(i=hid, j=out)

    # Set weights in correct order
    syn_output.w[:] = weights_3
    syn_output.layer = 2

    #print(syn_output.i[:], syn_output.j[:])
    #weights_into_output_1 = weights_3[1::3]



    neurons.run_regularly('''
        v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
        global_clock += 0.001
    ''', dt=0.001*ms)


    spikemon = SpikeMonitor(neurons)
    
    # neurons.v = 0
    # neurons.scheduled_time = 1e9 * second
    # neurons.global_clock = 0.0
    # neurons.sum = 0.0
    # neurons.spikes_received = 0.0

    run(5*ms)

    result = []

    for i in range(n_total):
        times = spikemon.spike_trains()[i]
        if len(times) > 0:
            result.append(round(times[0]/ms, 3))
        else:
            result.append(None)  # or some other placeholder like float('nan')
            
    return result

def real_outputs(var):
    return [round(var[-3 + i] - 2, 4) if var[-3 + i] is not None else 0.0 for i in range(3)]

# def calc_cost(outputs, desired_outputs):
#     return 0.5 * ((outputs - desired_outputs) ** 2)


# # --- Initialization ---
# w_1 = np.random.uniform(0.05, 0.95, size=4).tolist()      # input to hidden
# w_2 = np.random.uniform(-0.95, 0.95, size=40).tolist()     # hidden to next layer (10 hidden * 4 inputs)
# w_3 = np.random.uniform(-0.95, 0.95, size=30).tolist()     # next layer to output (10 hidden * 3 outputs)


# def softmax(x):
#     exps = np.exp(x - np.max(x))  # stability fix
#     return exps / np.sum(exps)

# def cross_entropy_loss(predicted_probs, target_one_hot):
#     return -np.sum(target_one_hot * np.log(predicted_probs + 1e-12))  # avoid log(0)

# def update_output_weights_softmax(hidden_activations, weights, target_one_hot, learning_rate=0.1):
#     """
#     hidden_activations: shape (n_hidden,)
#     weights: shape (n_classes, n_hidden)
#     target_one_hot: shape (n_classes,) - one-hot encoded desired output
#     """
#     # Forward pass
#     logits = weights @ hidden_activations  # shape (n_classes,)
#     probs = softmax(logits)                # shape (n_classes,)

#     # Compute loss (optional, for monitoring)
#     loss = cross_entropy_loss(probs, target_one_hot)

#     # Compute gradient of loss w.r.t. logits
#     grad_logits = probs - target_one_hot  # shape (n_classes,)

#     # Compute gradient of loss w.r.t. weights
#     grad_weights = np.outer(grad_logits, hidden_activations)  # shape (n_classes, n_hidden)

#     # Update weights
#     weights -= learning_rate * grad_weights

#     # Optional: clip to avoid explosion
#     weights = np.clip(weights, -5.0, 5.0)

#     return weights, probs, loss

# # # --- Target Output Generator ---
# # def make_target_output(label, high=5.0, low=0.05):
# #     target = [low, low, low]
# #     target[label] = high
# #     return target

# # # --- Softmax Function 
# # def softmax(x):
# #     e_x = np.exp(x - np.max(x))
# #     return e_x / np.sum(e_x)


# # def cross_entropy_loss(predicted_probs, target_one_hot):
# #     return -np.sum(target_one_hot * np.log(predicted_probs + 1e-12))  # avoid log(0)


# # # --- Update Weights into Output Neuron ---
# # def update_weights_input_neuron(hidden_activations, weights, actual_output, desired_output, learning_rate=0.1):
# #     error = actual_output - desired_output
# #     gradients = error * hidden_activations

# #     signs = np.sign(gradients)
# #     sign_str = ', '.join([f"{g:+.3f}" for g in gradients])
# #     print(f"Gradient signs: [{sign_str}] (Error: {error:+.3f})")
# #     print("Hidden activations:", hidden_activations)
# #     print("Errors:", actual_output - desired_output)

    
# #     updated_weights = weights - learning_rate * gradients
# #     updated_weights = np.clip(updated_weights, -5.0, 5.0)
# #     return updated_weights





# # # --- Update Hidden Neuron Weights ---
# # def update_single_hidden_neuron_weights(input_activations, weights_to_hidden, downstream_weights, output_errors, hidden_activation, learning_rate=0.1):
# #     propagated_error = np.dot(output_errors, downstream_weights)
# #     derivative = hidden_activation * (1 - hidden_activation)
# #     local_gradient = propagated_error * derivative
# #     gradients = local_gradient * input_activations
# #     updated_weights = weights_to_hidden - learning_rate * gradients
# #     updated_weights = np.clip(updated_weights, -0.95, 0.95)
# #     return updated_weights

# # # --- Update Hidden Layer ---
# # def update_hidden_layer_weights(spike_times, output_errors):
# #     lr = 0.1
# #     input_activations = np.array(spike_times[:4])
# #     hidden_activations = np.array(spike_times[4:14])

# #     for i in range(10):
# #         start_idx = i * 4
# #         end_idx = (i + 1) * 4

# #         weights_to_hidden_i = w_2[start_idx:end_idx]
# #         hidden_activation = hidden_activations[i]

# #         downstream_weights = np.array([
# #             w_3[i * 3 + 0],
# #             w_3[i * 3 + 1],
# #             w_3[i * 3 + 2],
# #         ])

# #         updated_weights = update_single_hidden_neuron_weights(
# #             input_activations,
# #             np.array(weights_to_hidden_i),
# #             downstream_weights,
# #             output_errors,
# #             hidden_activation,
# #             learning_rate=lr
# #         ) 

# #         w_2[start_idx:end_idx] = updated_weights


# def update_last_layer_weights(spike_times, desired_outputs, learning_rate=0.1):
#     """
#     Update output weights using softmax + cross-entropy.
#     `w_3`: shape 30 (flattened 10 hidden * 3 outputs)
#     """
#     hidden_activations = np.array(spike_times[4:14])   # 10 hidden neurons
#     actual_outputs = np.array(spike_times[-3:])        # old output activations (can ignore for this method)

#     # --- 1. Build weight matrix (3 classes x 10 hidden units)
#     weights = np.zeros((3, 10))
#     for out_idx in range(3):
#         weights[out_idx] = w_3[out_idx::3]

#     # --- 2. Forward Pass: logits and softmax
#     logits = weights @ hidden_activations               # shape (3,)
#     probs = softmax(logits)

#     # --- 3. Loss (optional: monitor training)
#     loss = cross_entropy_loss(probs, desired_outputs)

#     # --- 4. Gradient: dL/dz = softmax_output - target
#     grad_logits = probs - np.array(desired_outputs)     # shape (3,)

#     # --- 5. Gradient: dL/dW = outer(grad_logits, hidden_activations)
#     grad_weights = np.outer(grad_logits, hidden_activations)  # shape (3, 10)

#     # --- 6. Weight Update
#     weights -= learning_rate * grad_weights

#     # --- 7. Clip (optional)
#     weights = np.clip(weights, -5.0, 5.0)

#     # --- 8. Flatten back into w_3 list (stride write-back)
#     for out_idx in range(3):
#         for i in range(10):
#             w_3[out_idx + i * 3] = weights[out_idx, i]

#     # --- 9. Optional: Debug prints
#     grad_signs = np.sign(grad_weights)
#     for i in range(3):
#         sign_str = ', '.join(f"{v:+.3f}" for v in grad_weights[i])
#         print(f"Gradient signs (class {i}): [{sign_str}] (Error: {grad_logits[i]:+.3f})")
#     print("Hidden activations:", hidden_activations)
#     print("Softmax probabilities:", probs)
#     print("Cross-entropy loss:", loss)


# # --- Update Output Layer ---
# # def update_last_layer_weights(spike_times, desired_outputs, learning_rate=0.1):
# #     hidden_activations = np.array(spike_times[4:14])
# #     actual_outputs = np.array(spike_times[-3:])

# #     weights_0 = np.array(w_3[0::3])


    
# #     weights_1 = np.array(w_3[1::3])
# #     weights_2 = np.array(w_3[2::3])

# #     updated_0 = update_weights_input_neuron(hidden_activations, weights_0, actual_outputs[0], desired_outputs[0], learning_rate)
# #     updated_1 = update_weights_input_neuron(hidden_activations, weights_1, actual_outputs[1], desired_outputs[1], learning_rate)
# #     updated_2 = update_weights_input_neuron(hidden_activations, weights_2, actual_outputs[2], desired_outputs[2], learning_rate)

# #     w_3[0::3] = updated_0
# #     w_3[1::3] = updated_1
# #     w_3[2::3] = updated_2


# # def update_weights_input_neuron(hidden_activations, weights, actual_output, desired_output, learning_rate=0.1):

# #     """
# #     Update all weights going into a single output neuron using gradient descent.

# #     Args:
# #         hidden_activations (np.ndarray): Activations from hidden neurons (shape: [n_hidden]).
# #         weights (np.ndarray): Current weights into the output neuron (shape: [n_hidden]).
# #         actual_output (float): Current output of this neuron.
# #         desired_output (float): Target output for this neuron.
# #         learning_rate (float): Learning rate.

# #     Returns:
# #         np.ndarray: Updated weights (shape: [n_hidden]).
# #     """
# #     error = actual_output - desired_output
# #     gradients = error * hidden_activations
# #     updated_weights = weights - learning_rate * gradients
# #     updated_weights = np.clip(updated_weights, -0.95, 0.95)
# #     return updated_weights


# # def update_single_hidden_neuron_weights(input_activations, weights_to_hidden, downstream_weights, output_errors, hidden_activation, learning_rate=0.1):
# #     """
# #     Updates the weights into a single hidden neuron.

# #     Args:
# #         input_activations (np.ndarray): Input activations (shape: [n_input]).
# #         weights_to_hidden (np.ndarray): Weights into this hidden neuron (shape: [n_input]).
# #         downstream_weights (np.ndarray): Weights from this hidden neuron to each output neuron (shape: [n_output]).
# #         output_errors (np.ndarray): Errors at each output neuron (shape: [n_output]).
# #         hidden_activation (float): Activation value of this hidden neuron.
# #         learning_rate (float): Learning rate.

# #     Returns:
# #         np.ndarray: Updated weights for this hidden neuron (shape: [n_input]).
# #     """
# #     propagated_error = np.dot(output_errors, downstream_weights)
# #     derivative = hidden_activation * (1 - hidden_activation)
# #     local_gradient = propagated_error * derivative
# #     gradients = local_gradient * input_activations
# #     updated_weights = weights_to_hidden - learning_rate * gradients
# #     updated_weights = np.clip(updated_weights, -0.95, 0.95)
# #     return updated_weights


# # def update_hidden_layer_weights(spike_times, output_errors):
# #     lr = 0.1 # learning rate will need to universalize later

# #     input_activations = np.array(spike_times[:4])
# #     hidden_activations = np.array(spike_times[4:14])

# #     for i in range(10):  # 10 hidden neurons
# #         start_idx = i * 4
# #         end_idx = (i + 1) * 4

# #         weights_to_hidden_i = w_2[start_idx:end_idx]
# #         hidden_activation = hidden_activations[i]

# #         # FIXED: Correct indexing for downstream weights
# #         # Each hidden neuron i connects to all 3 outputs with weights at positions:
# #         # i*3, i*3+1, i*3+2
# #         downstream_weights = np.array([
# #             w_3[i * 3 + 0],  # weight from hidden neuron i to output 0
# #             w_3[i * 3 + 1],  # weight from hidden neuron i to output 1  
# #             w_3[i * 3 + 2],  # weight from hidden neuron i to output 2
# #         ])
        
# #         # Debug: Print the indices being used
# #         #print(f"  Weight indices for neuron {i}: [{i*3}, {i*3+1}, {i*3+2}]")
# #         #print(f"  w_3 values at those indices: {w_3[i*3:i*3+3]}")

# #         propagated_error = np.dot(output_errors, downstream_weights)
# #         derivative = hidden_activation * (1 - hidden_activation)
# #         local_gradient = propagated_error * derivative
# #         gradients = local_gradient * input_activations

# #         # print(f"\nHidden Neuron {i}")
# #         # print(f"  Hidden Activation: {hidden_activation}")
# #         # print(f"  Downstream Weights: {downstream_weights}")
# #         # print(f"  Propagated Error: {propagated_error}")
# #         # print(f"  Local Gradient: {local_gradient}")
# #         # print(f"  Gradients: {gradients}")

# #         updated_weights = weights_to_hidden_i - lr * gradients
# #         updated_weights = np.clip(updated_weights, -0.95, 0.95)
# #         w_2[start_idx:end_idx] = updated_weights

# #     # print("hidden layer weights updated")


# # def update_last_layer_weights(spike_times, desired_outputs):

# #     lr = 0.1 # will make more global later
    
# #     var = spike_times
# #     hidden_activations = np.array(var[4:14])


# #     weights_to_output_0 = w_3[0::3]
# #     weights_to_output_1 = w_3[1::3]
# #     weights_to_output_2 = w_3[2::3]

# #     actual_output_0 = var[-3]
# #     actual_output_1 = var[-2]
# #     actual_output_2 = var[-1]


# #     desired_output_0 = desired_outputs[0]
# #     desired_output_1 = desired_outputs[1]
# #     desired_output_2 = desired_outputs[2]


# #     new_weights_0 = update_weights_input_neuron(
# #         hidden_activations,
# #         weights_to_output_0,
# #         actual_output_0,
# #         desired_output_0,
# #         learning_rate=lr
# #     )

# #     new_weights_1 = update_weights_input_neuron(
# #         hidden_activations,
# #         weights_to_output_1,
# #         actual_output_1,
# #         desired_output_1,
# #         learning_rate=lr
# #     )

# #     new_weights_2 = update_weights_input_neuron(
# #         hidden_activations,
# #         weights_to_output_2,
# #         actual_output_2,
# #         desired_output_2,
# #         learning_rate=lr
# #     )

# #     w_3[0::3] = new_weights_0
# #     w_3[1::3] = new_weights_1
# #     w_3[2::3] = new_weights_2

# #     print("weights updated")



In [ ]:
import numpy as np
import logging

logging.getLogger('brian2').setLevel(logging.ERROR)

# inputs = [0.1, 0.2, 0.5, 0.9]
# wanted_output = [2.1, 2.1, 2.9]

# Initialize weights if not already
# w_1 = np.random.uniform(0.05, 0.95, size=4)
# w_2 = np.random.uniform(-0.95, 0.95, size=40)
# w_3 = np.random.uniform(-0.95, 0.95, size=30)

def real_outputs(var):
    return [round(var[-3 + i] - 2, 4) if var[-3 + i] is not None else 0.0 for i in range(3)]

def get_output_errors(var, wanted_output):
    return np.array([var[-3 + i] - wanted_output[i] for i in range(3)])


# print(w_2)
# for i in range(5):
#     try:
#         var = run_Urd(inputs, w_1, w_2, w_3)

#         print(f"Iter {i} Output:", real_outputs(var))

#         update_last_layer_weights(var, wanted_output)

#         # Compute error for hidden layer backprop
#         output_errors = get_output_errors(var, wanted_output)

#         update_hidden_layer_weights(var, output_errors)

#     except Exception as e:
#         print(f"Error in iteration {i}: {e}")

# # Final output
# print("Final Output:", real_outputs(var))
# print(w_2)


In [4]:
import numpy as np

def softmax(x):
    """Numerically stable softmax"""
    exps = np.exp(x - np.max(x))
    return exps / np.sum(exps)

def cross_entropy_loss(predicted_probs, target_one_hot):
    """Cross-entropy loss with numerical stability"""
    return -np.sum(target_one_hot * np.log(predicted_probs + 1e-12))

def sigmoid(x):
    """Sigmoid activation function"""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def sigmoid_derivative(x):
    """Derivative of sigmoid function"""
    s = sigmoid(x)
    return s * (1 - s)

class SNNBackpropagation:
    def __init__(self, w_1, w_2, w_3, learning_rate=0.1):
        """
        Initialize backpropagation for SNN
        w_1: input weights (4,) - weights from input to first layer
        w_2: hidden weights (40,) - weights from input (4) to hidden (10) - flattened
        w_3: output weights (30,) - weights from hidden (10) to output (3) - flattened
        """
        self.w_1 = np.array(w_1)
        self.w_2 = np.array(w_2).reshape(10, 4)  # 10 hidden neurons, 4 inputs each
        self.w_3 = np.array(w_3).reshape(3, 10)  # 3 output neurons, 10 hidden inputs each
        self.lr = learning_rate
        
    def forward_pass(self, spike_times):
        """
        Forward pass to compute activations
        spike_times: [input_spikes(4), hidden_spikes(10), output_spikes(3)]
        """
        self.input_activations = np.array(spike_times[:4])
        self.hidden_activations = np.array(spike_times[4:14])
        self.output_activations = np.array(spike_times[-3:])
        
        # Compute logits and probabilities for output layer
        self.output_logits = self.w_3 @ self.hidden_activations
        self.output_probs = softmax(self.output_logits)
        
        return self.output_probs
    
    def backward_pass(self, target_one_hot):
        """
        Backward pass to compute gradients and update weights
        target_one_hot: one-hot encoded target output [0, 1, 0] format
        """
        # === OUTPUT LAYER GRADIENTS ===
        # Gradient of loss w.r.t. output logits (softmax + cross-entropy)
        output_grad = self.output_probs - target_one_hot  # shape (3,)
        
        # Gradient of loss w.r.t. output weights
        grad_w3 = np.outer(output_grad, self.hidden_activations)  # shape (3, 10)
        
        # === HIDDEN LAYER GRADIENTS ===
        # Backpropagate error to hidden layer
        hidden_error = self.w_3.T @ output_grad  # shape (10,)
        
        # For spike-based networks, we need to approximate the derivative
        # Using sigmoid approximation for spike timing derivative
        hidden_grad = hidden_error * sigmoid_derivative(self.hidden_activations)  # shape (10,)
        
        # Gradient of loss w.r.t. hidden weights
        grad_w2 = np.outer(hidden_grad, self.input_activations)  # shape (10, 4)
        
        # === INPUT LAYER GRADIENTS (if needed) ===
        # Backpropagate to input layer (usually not updated in classification)
        input_error = self.w_2.T @ hidden_grad  # shape (4,)
        input_grad = input_error * sigmoid_derivative(self.input_activations)
        
        # === WEIGHT UPDATES ===
        self.w_3 -= self.lr * grad_w3
        self.w_2 -= self.lr * grad_w2
        # self.w_1 -= self.lr * input_grad  # Uncomment if you want to update input weights
        
        # Clip weights to prevent explosion
        self.w_3 = np.clip(self.w_3, -5.0, 5.0)
        self.w_2 = np.clip(self.w_2, -5.0, 5.0)
        self.w_1 = np.clip(self.w_1, -5.0, 5.0)
        
        return {
            'output_grad': output_grad,
            'hidden_grad': hidden_grad,
            'input_grad': input_grad,
            'grad_w3': grad_w3,
            'grad_w2': grad_w2
        }
    
    def get_flattened_weights(self):
        """Return weights in original flattened format for compatibility"""
        return {
            'w_1': self.w_1.tolist(),
            'w_2': self.w_2.flatten().tolist(),
            'w_3': self.w_3.flatten().tolist()
        }
    
    def train_step(self, spike_times, target_one_hot, verbose=True):
        """Complete training step: forward + backward + diagnostics"""
        # Forward pass
        probs = self.forward_pass(spike_times)
        loss = cross_entropy_loss(probs, target_one_hot)
        
        # Backward pass
        gradients = self.backward_pass(target_one_hot)
        
        # Diagnostics
        if verbose:
            print(f"Input activations: {self.input_activations}")
            print(f"Hidden activations: {self.hidden_activations}")
            print(f"Output probabilities: {probs}")
            print(f"Cross-entropy loss: {loss}")
            
            # Print gradient information
            print(f"Output gradients: {gradients['output_grad']}")
            print(f"Hidden gradients mean: {np.mean(np.abs(gradients['hidden_grad'])):.4f}")
            print(f"Weight update magnitudes:")
            print(f"  W3: {np.mean(np.abs(gradients['grad_w3'])):.4f}")
            print(f"  W2: {np.mean(np.abs(gradients['grad_w2'])):.4f}")
        
        return loss, probs

# Enhanced training function
def train_snn_with_full_backprop(w_1, w_2, w_3, training_data, epochs=100, learning_rate=0.1):
    """
    Train SNN with full backpropagation
    
    training_data: list of (spike_times, target_label) tuples
    spike_times: result from run_Urd function
    target_label: integer class label (0, 1, or 2)
    """
    # Initialize backprop trainer
    trainer = SNNBackpropagation(w_1, w_2, w_3, learning_rate)
    
    loss_history = []
    
    for epoch in range(epochs):
        epoch_loss = 0
        correct_predictions = 0
        
        print(f"\n=== Epoch {epoch} ===")
        
        for i, (spike_times, target_label) in enumerate(training_data):
            # Convert label to one-hot
            target_one_hot = np.zeros(3)
            target_one_hot[target_label] = 1.0
            
            # Handle None values in spike_times (neurons that didn't spike)
            processed_spike_times = []
            for st in spike_times:
                if st is None:
                    processed_spike_times.append(5.0)  # Late spike time for non-spiking neurons
                else:
                    processed_spike_times.append(st)
            
            print(f"\nSample {i}, Target class: {target_label}")
            print(f"Spike times: {processed_spike_times}")
            
            # Train on this sample
            loss, probs = trainer.train_step(processed_spike_times, target_one_hot, verbose=True)
            
            # Track metrics
            epoch_loss += loss
            predicted_class = np.argmax(probs)
            if predicted_class == target_label:
                correct_predictions += 1
                
            print(f"Predicted class: {predicted_class}, Correct: {predicted_class == target_label}")
        
        # Epoch summary
        avg_loss = epoch_loss / len(training_data)
        accuracy = correct_predictions / len(training_data)
        loss_history.append(avg_loss)
        
        print(f"\nEpoch {epoch} Summary:")
        print(f"Average Loss: {avg_loss:.4f}")
        print(f"Accuracy: {accuracy:.2%}")
        
        # Early stopping if loss is very low
        if avg_loss < 0.01:
            print("Early stopping - loss converged")
            break
    
    # Return updated weights
    updated_weights = trainer.get_flattened_weights()
    return updated_weights, loss_history

# Example usage function
def example_training_loop():
    """Example of how to use the enhanced training"""
    
    # Your existing weight initialization
    w_1 = np.random.uniform(0.05, 0.95, size=4).tolist()
    w_2 = np.random.uniform(-0.95, 0.95, size=40).tolist()
    w_3 = np.random.uniform(-0.95, 0.95, size=30).tolist()
    
    # Example training data (you'll need to generate this with your run_Urd function)
    # training_data = [
    #     (spike_times_sample_1, target_class_1),
    #     (spike_times_sample_2, target_class_2),
    #     ...
    # ]
    
    # For demonstration, using dummy data
    training_data = [
        ([1.5, 2.0, 2.5, 3.0, 1.8, 2.2, 2.7, 3.2, 1.9, 2.3, 2.8, 3.3, 2.0, 2.4, 2.9], 0),
        ([2.0, 2.5, 3.0, 3.5, 2.3, 2.7, 3.2, 3.7, 2.4, 2.8, 3.3, 3.8, 2.5, 2.9, 3.4], 1),
        ([1.0, 1.5, 2.0, 2.5, 1.3, 1.7, 2.2, 2.7, 1.4, 1.8, 2.3, 2.8, 1.5, 1.9, 2.4], 2),
    ]
    
    # Train the network
    updated_weights, loss_history = train_snn_with_full_backprop(
        w_1, w_2, w_3, 
        training_data, 
        epochs=50, 
        learning_rate=0.1
    )
    
    print("\nFinal updated weights:")
    print(f"w_1: {len(updated_weights['w_1'])} elements")
    print(f"w_2: {len(updated_weights['w_2'])} elements") 
    print(f"w_3: {len(updated_weights['w_3'])} elements")
    
    return updated_weights, loss_history

# Integration with your existing code
def update_your_weights(spike_times, target_label, w_1, w_2, w_3, learning_rate=0.1):
    """
    Drop-in replacement for your current weight update function
    """
    # Convert target to one-hot
    target_one_hot = np.zeros(3)
    target_one_hot[target_label] = 1.0
    
    # Handle None values
    processed_spike_times = []
    for st in spike_times:
        if st is None:
            processed_spike_times.append(5.0)
        else:
            processed_spike_times.append(st)
    
    # Create trainer and perform one update step
    trainer = SNNBackpropagation(w_1, w_2, w_3, learning_rate)
    loss, probs = trainer.train_step(processed_spike_times, target_one_hot)
    
    # Return updated weights in original format
    updated_weights = trainer.get_flattened_weights()
    return updated_weights['w_1'], updated_weights['w_2'], updated_weights['w_3'], loss, probs

In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
from collections import defaultdict

# Your enhanced backpropagation class (from previous artifact)
class SNNBackpropagation:
    def __init__(self, w_1, w_2, w_3, learning_rate=0.1):
        """
        Initialize backpropagation for SNN
        w_1: input weights (4,) - weights from input to first layer
        w_2: hidden weights (40,) - weights from input (4) to hidden (10) - flattened
        w_3: output weights (30,) - weights from hidden (10) to output (3) - flattened
        """
        self.w_1 = np.array(w_1)
        self.w_2 = np.array(w_2).reshape(10, 4)  # 10 hidden neurons, 4 inputs each
        self.w_3 = np.array(w_3).reshape(3, 10)  # 3 output neurons, 10 hidden inputs each
        self.lr = learning_rate
        
    def softmax(self, x):
        """Numerically stable softmax"""
        exps = np.exp(x - np.max(x))
        return exps / np.sum(exps)

    def cross_entropy_loss(self, predicted_probs, target_one_hot):
        """Cross-entropy loss with numerical stability"""
        return -np.sum(target_one_hot * np.log(predicted_probs + 1e-12))

    def sigmoid(self, x):
        """Sigmoid activation function"""
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def sigmoid_derivative(self, x):
        """Derivative of sigmoid function"""
        s = self.sigmoid(x)
        return s * (1 - s)
        
    def forward_pass(self, spike_times):
        """
        Forward pass to compute activations
        spike_times: [input_spikes(4), hidden_spikes(10), output_spikes(3)]
        """
        self.input_activations = np.array(spike_times[:4])
        self.hidden_activations = np.array(spike_times[4:14])
        self.output_activations = np.array(spike_times[-3:])
        
        # Compute logits and probabilities for output layer
        self.output_logits = self.w_3 @ self.hidden_activations
        self.output_probs = self.softmax(self.output_logits)
        
        return self.output_probs
    
def spike_timing_derivative(x, w):
    """Derivative of the spike timing function"""
    if w >= 0:
        return (1 - w) * (x ** (-w))
    else:
        return (1 + w) * ((1 - x) ** (-w - 1))

def backward_pass(self, target_one_hot):
    """
    Backward pass to compute gradients and update weights
    """
    # === OUTPUT LAYER GRADIENTS ===
    # Gradient of loss w.r.t. output logits (softmax + cross-entropy)
    output_grad = self.output_probs - target_one_hot  # shape (3,)
    
    # === HIDDEN LAYER GRADIENTS ===
    # Backpropagate error to hidden layer
    hidden_error = self.w_3.T @ output_grad  # shape (10,)
    
    # For spike-based networks, use spike timing derivative
    hidden_grad = np.zeros_like(hidden_error)
    for i in range(len(hidden_error)):
        # Use spike timing derivative instead of sigmoid
        hidden_grad[i] = hidden_error[i] * spike_timing_derivative(
            self.hidden_activations[i], self.w_3[:, i]
        )
    
    # === INPUT LAYER GRADIENTS ===
    input_grad = np.zeros_like(self.input_activations)
    for i in range(len(self.input_activations)):
        input_grad[i] = (self.w_2.T @ hidden_grad)[i] * spike_timing_derivative(
            self.input_activations[i], self.w_2[:, i]
        )
    
    # === WEIGHT UPDATES ===
    # Update output weights
    grad_w3 = np.outer(output_grad, self.hidden_activations)
    self.w_3 -= self.lr * grad_w3
    
    # Update hidden weights
    grad_w2 = np.outer(hidden_grad, self.input_activations)
    self.w_2 -= self.lr * grad_w2
    
    # Clip weights
    self.w_3 = np.clip(self.w_3, -5.0, 5.0)
    self.w_2 = np.clip(self.w_2, -5.0, 5.0)
    
    return {
        'output_grad': output_grad,
        'hidden_grad': hidden_grad,
        'input_grad': input_grad,
        'grad_w3': grad_w3,
        'grad_w2': grad_w2
    }
    
    def get_flattened_weights(self):
        """Return weights in original flattened format for compatibility"""
        return {
            'w_1': self.w_1.tolist(),
            'w_2': self.w_2.flatten().tolist(),
            'w_3': self.w_3.flatten().tolist()
        }
    
    def train_step(self, spike_times, target_one_hot, verbose=False):
        """Complete training step: forward + backward + diagnostics"""
        # Forward pass
        probs = self.forward_pass(spike_times)
        loss = self.cross_entropy_loss(probs, target_one_hot)
        
        # Backward pass
        gradients = self.backward_pass(target_one_hot)
        
        # Diagnostics
        if verbose:
            print(f"Hidden activations: {self.hidden_activations}")
            print(f"Softmax probabilities: {probs}")
            print(f"Cross-entropy loss: {loss}")
            
            # Print gradient information for output layer (matching your original format)
            for i in range(3):
                grad_signs = gradients['grad_w3'][i]
                sign_str = ', '.join(f"{v:+.3f}" for v in grad_signs)
                print(f"Gradient signs (class {i}): [{sign_str}] (Error: {gradients['output_grad'][i]:+.3f})")
        
        return loss, probs

# Load and prepare Iris dataset
def prepare_iris_data():
    """Load and prepare Iris dataset"""
    # Load the Iris dataset
    iris = load_iris()
    X = iris.data
    y = iris.target

    def custom_scale(X, min_val=0.05, max_val=0.95):
        X_min = np.min(X, axis=0)
        X_max = np.max(X, axis=0)
        return min_val + (X - X_min) * (max_val - min_val) / (X_max - X_min)

    X_scaled = custom_scale(X)

    # Combine and shuffle (important!)
    combined = list(zip(X_scaled, y))
    np.random.seed(42)
    np.random.shuffle(combined)

    X_shuffled, y_shuffled = zip(*combined)

    # Split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_shuffled, y_shuffled, test_size=0.2, random_state=42
    )

    training_data = [list(x) for x in X_train]
    result_data = list(y_train)
    
    return training_data, result_data, X_test, y_test

# Initialize weights
def initialize_weights():
    """Initialize network weights"""
    w_1 = np.random.uniform(0.05, 0.95, size=4).tolist()
    w_2 = np.random.uniform(-0.95, 0.95, size=40).tolist()
    w_3 = np.random.uniform(-0.95, 0.95, size=30).tolist()
    return w_1, w_2, w_3

# Enhanced training function
def train_Urd_enhanced(training_data, result_data, w_1, w_2, w_3, loops=5, learning_rate=0.1):
    """
    Enhanced training function with full backpropagation
    """
    samples = len(training_data)
    print(f"Training on {samples} samples for {loops} epochs")
    
    # Initialize trainer
    trainer = SNNBackpropagation(w_1, w_2, w_3, learning_rate)
    
    # Training loop
    for epoch in range(loops):
        print(f"\n=== Epoch {epoch + 1}/{loops} ===")
        epoch_loss = 0
        correct_predictions = 0
        
        for i in range(samples):
            try:
                # Run SNN forward pass
                spike_times = run_Urd(training_data[i], 
                                    trainer.w_1.tolist(), 
                                    trainer.w_2.flatten().tolist(), 
                                    trainer.w_3.flatten().tolist())
                
                # Handle None values in spike_times (neurons that didn't spike)
                processed_spike_times = []
                for st in spike_times:
                    if st is None:
                        processed_spike_times.append(5.0)  # Late spike time for non-spiking neurons
                    else:
                        processed_spike_times.append(st)
                
                # Convert target to one-hot
                target_one_hot = np.zeros(3)
                target_one_hot[result_data[i]] = 1.0
                
                # Print current output (matching your original format)
                outputs = real_outputs(spike_times)
                print(f"Iter {i} Output: {outputs}")
                
                # Train step
                loss, probs = trainer.train_step(processed_spike_times, target_one_hot, verbose=True)
                
                # Track metrics
                epoch_loss += loss
                predicted_class = np.argmax(probs)
                if predicted_class == result_data[i]:
                    correct_predictions += 1
                
                print(f"Target: Class {result_data[i]}, Predicted: Class {predicted_class}")
                print("-" * 50)
                
            except Exception as e:
                print(f"Error in iteration {i}: {e}")
                continue
        
        # Epoch summary
        avg_loss = epoch_loss / samples
        accuracy = correct_predictions / samples
        print(f"\nEpoch {epoch + 1} Summary:")
        print(f"Average Loss: {avg_loss:.4f}")
        print(f"Training Accuracy: {accuracy:.2%}")
        print(f"Epoch {epoch + 1} completed")
        
        # Early stopping if accuracy is very high
        if accuracy > 0.95:
            print("Early stopping - high accuracy achieved")
            break
    
    print(f"\nTraining completed after {min(epoch + 1, loops)} epochs")
    
    # Return updated weights in original format
    final_weights = trainer.get_flattened_weights()
    return final_weights['w_1'], final_weights['w_2'], final_weights['w_3']

# Evaluation function
def evaluate_model(X_test, y_test, w_1, w_2, w_3):
    """Evaluate the trained model"""
    correct = 0
    total = 0
    class_correct = defaultdict(int)
    class_total = defaultdict(int)

    print("\n--- Evaluation on Test Data ---")
    for i, sample in enumerate(X_test):
        try:
            pred = run_Urd(sample.tolist(), w_1, w_2, w_3)
            outputs = real_outputs(pred)
            predicted_class = int(np.argmax(outputs))

            actual_class = int(y_test[i]) if isinstance(y_test[i], (int, np.integer)) else int(np.argmax(y_test[i]))

            if predicted_class == actual_class:
                correct += 1
                class_correct[actual_class] += 1
            class_total[actual_class] += 1
            total += 1

            print(f"Sample {i}: Predicted Class: {predicted_class}, Actual Class: {actual_class}, Outputs: {outputs}")
        except Exception as e:
            print(f"Error evaluating sample {i}: {e}")
            total += 1  # Still count it as a sample
            actual_class = int(y_test[i])
            class_total[actual_class] += 1

    # Final results
    print(f"\nOverall Accuracy: {correct}/{total} = {correct / total:.2%}")

    for cls in range(3):
        if class_total[cls] > 0:
            acc = class_correct[cls] / class_total[cls]
            print(f"Accuracy for Class {cls}: {class_correct[cls]}/{class_total[cls]} = {acc:.2%}")
        else:
            print(f"Class {cls} has no samples in test set.")
    
    return correct / total if total > 0 else 0

# Main training and evaluation pipeline
def main_training_pipeline():
    """Complete training and evaluation pipeline"""
    print("=== SNN Training with Enhanced Backpropagation ===")
    
    # Prepare data
    print("Loading and preparing Iris dataset...")
    training_data, result_data, X_test, y_test = prepare_iris_data()
    print(f"Training samples: {len(training_data)}")
    print(f"Test samples: {len(X_test)}")
    
    # Initialize weights
    print("\nInitializing weights...")
    w_1, w_2, w_3 = initialize_weights()
    print(f"Initial weights - w_1: {len(w_1)}, w_2: {len(w_2)}, w_3: {len(w_3)}")
    
    # Train the model
    print("\nStarting training...")
    w_1_trained, w_2_trained, w_3_trained = train_Urd_enhanced(
        training_data, result_data, w_1, w_2, w_3, 
        loops=3,  # Reduced for initial testing
        learning_rate=0.05  # Conservative learning rate
    )
    
    # Evaluate the model
    print("\nEvaluating trained model...")
    final_accuracy = evaluate_model(X_test, y_test, w_1_trained, w_2_trained, w_3_trained)
    
    # Save weights
    print("\nSaving trained weights...")
    np.savez('weights_checkpoint_enhanced.npz', 
             w1=w_1_trained, w2=w_2_trained, w3=w_3_trained)
    print("Weights saved to 'weights_checkpoint_enhanced.npz'")
    
    return w_1_trained, w_2_trained, w_3_trained, final_accuracy

# Usage example:
# Assuming you have run_Urd and real_outputs functions defined elsewhere
# w_1_final, w_2_final, w_3_final, accuracy = main_training_pipeline()

# For integration with your existing code, you can also use individual functions:

# Example usage with your existing setup:

# 1. Prepare data
training_data, result_data, X_test, y_test = prepare_iris_data()

# 2. Initialize or load weights
w_1, w_2, w_3 = initialize_weights()
# or load existing: weights = np.load('weights_checkpoint.npz'); w_1, w_2, w_3 = weights['w1'], weights['w2'], weights['w3']

# 3. Train with enhanced method
w_1, w_2, w_3 = train_Urd_enhanced(training_data, result_data, w_1, w_2, w_3, loops=1)

# 4. Evaluate
accuracy = evaluate_model(X_test, y_test, w_1, w_2, w_3)

# 5. Save
np.savez('weights_checkpoint_enhancedv_2cleaningrequired.npz', w1=w_1, w2=w_2, w3=w_3)

Training on 120 samples for 1 epochs

=== Epoch 1/1 ===
Iter 0 Output: [0.354, 0.5, 0.533]
Error in iteration 0: 'SNNBackpropagation' object has no attribute 'train_step'
Iter 1 Output: [0.483, 0.615, 0.663]
Error in iteration 1: 'SNNBackpropagation' object has no attribute 'train_step'
Iter 2 Output: [0.344, 0.49, 0.522]
Error in iteration 2: 'SNNBackpropagation' object has no attribute 'train_step'
Iter 3 Output: [0.314, 0.461, 0.488]
Error in iteration 3: 'SNNBackpropagation' object has no attribute 'train_step'
Iter 4 Output: [0.481, 0.615, 0.659]
Error in iteration 4: 'SNNBackpropagation' object has no attribute 'train_step'
Iter 5 Output: [0.417, 0.561, 0.596]
Error in iteration 5: 'SNNBackpropagation' object has no attribute 'train_step'
Iter 6 Output: [0.559, 0.677, 0.729]
Error in iteration 6: 'SNNBackpropagation' object has no attribute 'train_step'
Iter 7 Output: [0.541, 0.664, 0.719]
Error in iteration 7: 'SNNBackpropagation' object has no attribute 'train_step'
Iter 8 Outp

KeyboardInterrupt: 

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

def custom_scale(X, min_val=0.05, max_val=0.95):  # arelayd scaled
    X_min = np.min(X, axis=0)
    X_max = np.max(X, axis=0)
    return min_val + (X - X_min) * (max_val - min_val) / (X_max - X_min)

X_scaled = custom_scale(X)

# Normalize to [0.05, 0.95]
X_min = X.min(axis=0)
X_max = X.max(axis=0)
X_scaled = 0.05 + ((X - X_min) / (X_max - X_min)) * (0.95 - 0.05)

# Combine and shuffle (important!)
combined = list(zip(X_scaled, y))
np.random.seed(42)
np.random.shuffle(combined)

X_shuffled, y_shuffled = zip(*combined)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_shuffled, y_shuffled, test_size=0.2, random_state=42
)

training_data = [list(x) for x in X_train]
result_data = list(y_train)

# Step 3: Define training function (your original code)
def train_Urd(training_data, result_data, loops):
    samples = len(training_data)

    wanted_output = [[-.1, -.1, -.1] for _ in range(samples)]
    
    for i in range(samples):
        if result_data[i] == 0:
            wanted_output[i] = [3.0, 2.05, 2.05]
        elif result_data[i] == 1:
            wanted_output[i] = [2.05, 3.0, 2.05]
        elif result_data[i] == 2:
            wanted_output[i] = [2.05, 2.05, 3.0]
        else:
            print("PRoblemmm")

    for j in range(loops):
        for i in range(samples):
            try:
                var = run_Urd(training_data[i], w_1, w_2, w_3)

                print(f"Iter {i} Output:", real_outputs(var))

                update_last_layer_weights(var, wanted_output[i])

                # calc error for hidden layer backprop
                output_errors = get_output_errors(var, wanted_output[i])

                update_hidden_layer_weights(var, output_errors)

            except Exception as e:
                print(f"Error in iteration {i}: {e}")

        print(f"{j + 1} loop of training data completed")
    
    print(f"{loops} loops completed")


# Step 4: Train your model
train_Urd(training_data, result_data, loops=1)

from collections import defaultdict

correct = 0
total = 0
class_correct = defaultdict(int)
class_total = defaultdict(int)

print("\n--- Evaluation on Test Data ---")
for i, sample in enumerate(X_test):
    pred = run_Urd(sample.tolist(), w_1, w_2, w_3)
    outputs = real_outputs(pred)
    predicted_class = int(np.argmax(outputs))

    actual_class = int(y_test[i]) if isinstance(y_test[i], (int, np.integer)) else int(np.argmax(y_test[i]))

    if predicted_class == actual_class:
        correct += 1
        class_correct[actual_class] += 1
    class_total[actual_class] += 1
    total += 1

    print(f"Sample {i}: Predicted Class: {predicted_class}, Actual Class: {actual_class}, Outputs: {outputs}")

# Final results
print(f"\nOverall Accuracy: {correct}/{total} = {correct / total:.2%}")

for cls in range(3):
    if class_total[cls] > 0:
        acc = class_correct[cls] / class_total[cls]
        print(f"Accuracy for Class {cls}: {class_correct[cls]}/{class_total[cls]} = {acc:.2%}")
    else:
        print(f"Class {cls} has no samples in test set.")

np.savez('weights_checkpoint.npz', w1=w_1, w2=w_2, w3=w_3)
# when weights was all 2.05 and 2.95 all values seemed give out that was either all for A or B


Iter 0 Output: [1.168, 1.167, 1.171]
Gradient signs (class 0): [-5.521, -5.521, -5.521, -5.521, -5.521, -5.521, -5.521, -5.521, -5.521, -5.521] (Error: -2.820)
Gradient signs (class 1): [-3.764, -3.764, -3.764, -3.764, -3.764, -3.764, -3.764, -3.764, -3.764, -3.764] (Error: -1.922)
Gradient signs (class 2): [-2.659, -2.659, -2.659, -2.659, -2.659, -2.659, -2.659, -2.659, -2.659, -2.659] (Error: -1.358)
Hidden activations: [1.958 1.958 1.958 1.958 1.958 1.958 1.958 1.958 1.958 1.958]
Softmax probabilities: [0.18016129 0.12773607 0.69210263]
Cross-entropy loss: 10.114618924065724
Iter 1 Output: [1.065, 1.065, 1.065]
Gradient signs (class 0): [-3.451, -3.451, -3.451, -3.451, -3.451, -3.451, -3.451, -3.451, -3.451, -3.451] (Error: -1.740)
Gradient signs (class 1): [-5.382, -5.382, -5.382, -5.382, -5.382, -5.382, -5.382, -5.382, -5.382, -5.382] (Error: -2.714)
Gradient signs (class 2): [-3.264, -3.264, -3.264, -3.264, -3.264, -3.264, -3.264, -3.264, -3.264, -3.264] (Error: -1.646)
Hidden ac

KeyboardInterrupt: 

In [ ]:
# training_data = [[0.1, 0.5, 0.1, 0.75], [0.3, 0.5, 0.21, 0.65]]
# result_data = [0, 1]

# def train_Urd(training_data, result_data, loops):
#     samples = len(training_data)

#     wanted_output = [[-.1, -.1, -.1] for _ in range(samples)]
    
#     for i in range(samples):
#         if result_data[i] == 0:
#             wanted_output[i] = [0.95, 0.05, 0.05]
#         elif result_data[i] == 1:
#             wanted_output[i] = [0.05, 0.95, 0.05]
#         elif result_data[i] == 2:
#             wanted_output[i] = [0.05, 0.05, 0.95]

#     for j in range(loops):
#         for i in range(samples):
#             try:
#                 var = run_Urd(training_data[i], w_1, w_2, w_3)

#                 print(f"Iter {i} Output:", real_outputs(var))

#                 update_last_layer_weights(var, wanted_output[i])

#                 # calc error for hidden layer backprop
#                 output_errors = get_output_errors(var, wanted_output[i])

#                 update_hidden_layer_weights(var, output_errors)

#             except Exception as e:
#                 print(f"Error in iteration {i}: {e}")

#         print(f"{j + 1} loop of training data completed")
    
#     print(f"{loops} loops completed")



# train_Urd(training_data, result_data, 1)